# Kotlin Coroutines: Basic Examples in a Notebook

This notebook demonstrates essential coroutine patterns using Kotlin in a JetBrains Notebook.

## Index
- [Setup: common imports](#setup-common-imports)
- [1) launch with runBlocking](#1-launch-with-runblocking)
- [2) async/await for concurrent decomposition](#2-asyncawait-for-concurrent-decomposition)
- [3) Structured concurrency with coroutineScope](#3-structured-concurrency-with-coroutinescope)
- [4) Cancellation and timeouts](#4-cancellation-and-timeouts)
- [5) Switching threads with withContext and Dispatchers](#5-switching-threads-with-withcontext-and-dispatchers)
- [6) Flow: cold asynchronous stream](#6-flow-cold-asynchronous-stream)
- [7) Callback hell: what it is and how coroutines help](#7-callback-hell-what-it-is-and-how-coroutines-help)
- [8) Dispatchers explained: choosing where coroutines run](#8-dispatchers-explained-choosing-where-coroutines-run)
- [9) Structured concurrency in Kotlin](#9-structured-concurrency-in-kotlin)
- [10) What is a Job in Kotlin coroutines?](#10-what-is-a-job-in-kotlin-coroutines)
- [11) Dispatcher.Main.immediate: what it is and when to use it](#11-dispatchermainimmediate-what-it-is-and-when-to-use-it)
- [Appendix: The Android event loop (Looper/MessageQueue) and coroutines](#appendix-the-android-event-loop-loopermessagequeue-and-coroutines)


In [19]:
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.8.1")

## Setup: common imports
To keep the notebook tidy, we import commonly used coroutine APIs up front.


In [20]:
import kotlinx.coroutines.*
import kotlinx.coroutines.flow.*
import kotlin.system.measureTimeMillis
import java.util.concurrent.Executors


## 1) launch with runBlocking
Use runBlocking in notebooks (or main functions) to bridge blocking and suspending worlds.
launch starts a new coroutine that runs concurrently and returns a Job.


In [21]:
runBlocking {
    val job = launch {
        repeat(3) { i ->
            delay(300)
            println("[launch] Tick $i")
        }
    }
    println("[main] Waiting for child coroutine...")
    job.join()
    println("[main] Done.")
}


[main] Waiting for child coroutine...
[launch] Tick 0
[launch] Tick 1
[launch] Tick 2
[main] Done.


## 2) async/await for concurrent decomposition
async returns a Deferred; await waits for the result.
Both children run concurrently here. We also measure elapsed time.


In [4]:
runBlocking {
    val elapsed = measureTimeMillis {
        val one = async { delay(500); 40 }
        val two = async { delay(500); 2 }
        val result = one.await() + two.await()
        println("[async] Result = $result")
    }
    println("[async] Completed in ${elapsed} ms")
}


[async] Result = 42
[async] Completed in 503 ms


## 3) Structured concurrency with coroutineScope
coroutineScope waits for all its children to complete, ensuring proper structure and lifecycle.


In [5]:
runBlocking {
    coroutineScope {
        launch {
            delay(200)
            println("[scope] Task from nested launch")
        }
        println("[scope] Inside coroutineScope block")
    }
    println("[scope] coroutineScope is over (all children completed)")
}



[scope] Inside coroutineScope block
[scope] Task from nested launch
[scope] coroutineScope is over (all children completed)


## 4) Cancellation and timeouts
Jobs are cancellable. You can cancel a Job explicitly or use timeouts.


In [6]:
// Explicit cancel
runBlocking {
    val job = launch {
        try {
            repeat(10) { i ->
                println("[cancel] I'm sleeping $i ...")
                delay(200)
            }
        } finally {
            println("[cancel] finally: cleanup if needed")
        }
    }
    delay(700)
    println("[cancel] main: Tired of waiting!")
    job.cancelAndJoin()
    println("[cancel] main: Done.")
}

// Timeout
runBlocking {
    val result = withTimeoutOrNull(500) {
        repeat(10) { _ ->
            delay(200)
        }
        "Finished"
    }
    println("[timeout] result = $result")
}


[cancel] I'm sleeping 0 ...
[cancel] I'm sleeping 1 ...
[cancel] I'm sleeping 2 ...
[cancel] I'm sleeping 3 ...
[cancel] main: Tired of waiting!
[cancel] finally: cleanup if needed
[cancel] main: Done.
[timeout] result = null


## 5) Switching threads with withContext and Dispatchers
Use withContext to shift to a thread pool appropriate for the work.


In [7]:
fun cpuIntensive(): Long {
    // Dummy CPU work
    return (1..1_000_000).fold(0L) { acc, i -> acc + (i % 7) }
}

runBlocking {
    val result = withContext(Dispatchers.Default) {
        cpuIntensive()
    }
    println("[withContext] Computed on Default dispatcher: $result")
}


[withContext] Computed on Default dispatcher: 2999998


## 6) Flow: cold asynchronous stream
Flow builds values lazily and emits asynchronously. Collect to consume.


In [8]:
fun numbers(): Flow<Int> = flow {
    for (i in 1..5) {
        delay(100)
        emit(i)
    }
}

runBlocking {
    numbers()
        .map { it * it }
        .onEach { println("[flow] Emitting $it") }
        .collect()
}


[flow] Emitting 1
[flow] Emitting 4
[flow] Emitting 9
[flow] Emitting 16
[flow] Emitting 25


## 7) Callback hell: what it is and how coroutines help

Callback hell is what happens when asynchronous code is expressed as nested callbacks that grow to the right, making control flow hard to read, reason about, and handle errors for. Symptoms include:
- Deeply nested lambdas ("pyramid of doom")
- Inversion of control (your logic is scattered across callbacks)
- Hard-to-compose error handling and cancellation

Why it happens: many async APIs expose callback-style interfaces (success/failure listeners) instead of suspend functions. Chaining multiple async operations leads to nesting.

How coroutines help: coroutines let you write asynchronous code in a sequential style using suspend functions, structured concurrency, and async/await. This keeps the code flat, composable, and testable while still being non-blocking.

Example (illustrative):

```kotlin
// Without coroutines: nested callbacks (pseudo-API)
fun fetchUser(id: String, onSuccess: (User) -> Unit, onError: (Throwable) -> Unit)
fun fetchPosts(user: User, onSuccess: (List<Post>) -> Unit, onError: (Throwable) -> Unit)
fun render(posts: List<Post>)

fetchUser("42", { user ->
    fetchPosts(user, { posts ->
        try {
            render(posts)
        } catch (t: Throwable) {
            println("render error: ${t.message}")
        }
    }, { e ->
        println("posts error: ${e.message}")
    })
}, { e ->
    println("user error: ${e.message}")
})
```

With coroutines, convert callback-style APIs to suspend functions (often provided by libraries or via wrappers), then compose sequentially:

```kotlin
// With coroutines (illustrative):
suspend fun fetchUser(id: String): User
suspend fun fetchPosts(user: User): List<Post>

runBlocking {
    try {
        val user = fetchUser("42")           // suspends, no thread blocked
        val posts = fetchPosts(user)           // sequential, still non-blocking
        render(posts)                          // regular call
    } catch (e: Throwable) {
        println("error: ${e.message}")
    }
}
```

Parallelizing independent work stays readable:

```kotlin
runBlocking {
    val userDeferred = async { fetchUser("42") }
    val settingsDeferred = async { fetchSettings("42") }
    val user = userDeferred.await()
    val settings = settingsDeferred.await()
    apply(user, settings)
}
```

Takeaways:
- Prefer suspend APIs over raw callbacks when available.
- Wrap callback-based APIs into suspend functions (e.g., using suspendCancellableCoroutine) to escape callback hell.
- Use structured concurrency (scope, Job hierarchy) for lifecycle, error propagation, and cancellation.


## 8) Dispatchers explained: choosing where coroutines run

Dispatchers define the thread(s) a coroutine uses. Pick the dispatcher based on the nature of work:

- Dispatchers.Default — CPU-bound work. Backed by a shared pool sized by CPU cores. Use for computations, parsing, etc.
- Dispatchers.IO — Blocking I/O (files, sockets, DB drivers). Larger thread pool tuned for blocking operations; prefer this when you must block a thread.
- Dispatchers.Main — Main UI thread (Android/JavaFX/Swing). Requires an extra artifact (e.g., kotlinx-coroutines-android, -javafx, or -swing) and only exists in those runtimes. Not used in this notebook.
- Dispatchers.Unconfined — Starts in the current call-frame thread and resumes in the thread of the next suspension. Useful for tests or advanced cases; avoid for general app code because it’s not confined to a specific thread.
- Custom dispatcher — Create from an Executor or thread pool when you need dedicated threads. Remember to close it.

Tips:
- Use withContext(dispatcher) to switch context for a block.
- Avoid newSingleThreadContext (deprecated). Prefer Executors.newSingleThreadExecutor().asCoroutineDispatcher().
- Limit parallelism: Dispatchers.Default/IO can be sliced with limitedParallelism(N) for backpressure.

### See it in action: Default vs IO


In [9]:
runBlocking {
    val d = withContext(Dispatchers.Default) {
        Thread.currentThread().name
    }
    val io = withContext(Dispatchers.IO) {
        Thread.currentThread().name
    }
    println("[dispatcher] Default on: $d")
    println("[dispatcher] IO on: $io")
}


[dispatcher] Default on: DefaultDispatcher-worker-1
[dispatcher] IO on: DefaultDispatcher-worker-1


### Unconfined dispatcher behavior
Unconfined starts in the caller thread, but after suspension it resumes in whatever thread the suspension resumes on (e.g., a scheduler). This can change between calls.


In [10]:
runBlocking {
    println("[unconfined] start on: ${Thread.currentThread().name}")
    launch(Dispatchers.Unconfined) {
        println("[unconfined] before delay: ${Thread.currentThread().name}")
        delay(100)
        println("[unconfined] after delay: ${Thread.currentThread().name}")
    }.join()
}


[unconfined] start on: Execution of code 'import kotlinx.corou...'
[unconfined] before delay: Execution of code 'import kotlinx.corou...'
[unconfined] after delay: kotlinx.coroutines.DefaultExecutor


### Custom single-thread dispatcher (via Executor)
Sometimes you need strict confinement (e.g., a legacy library requiring single-threaded access).


In [22]:
import java.util.concurrent.Executors
import kotlin.concurrent.thread

val single = Executors.newSingleThreadExecutor().asCoroutineDispatcher()
try {
    runBlocking(single) {
        repeat(3) { i ->
            println("[single] $i on ${Thread.currentThread().name}")
            delay(50)
        }
    }
} finally {
    single.close() // always close custom dispatchers
}


[single] 0 on pool-2-thread-1
[single] 1 on pool-2-thread-1
[single] 2 on pool-2-thread-1


### Limiting parallelism
Use limitedParallelism to cap concurrency on a shared pool for a specific sub-system.


In [12]:
runBlocking {
    val limitedIO = Dispatchers.IO.limitedParallelism(2)
    val jobs = List(5) { idx ->
        launch(limitedIO) {
            println("[limitedIO] start $idx on ${Thread.currentThread().name}")
            delay(200)
            println("[limitedIO] end   $idx on ${Thread.currentThread().name}")
        }
    }
    jobs.forEach { it.join() }
}



[limitedIO] start 1 on DefaultDispatcher-worker-2
[limitedIO] start 0 on DefaultDispatcher-worker-1
[limitedIO] start 3 on DefaultDispatcher-worker-1
[limitedIO] start 4 on DefaultDispatcher-worker-1
[limitedIO] start 2 on DefaultDispatcher-worker-2
[limitedIO] end   1 on DefaultDispatcher-worker-2
[limitedIO] end   3 on DefaultDispatcher-worker-5
[limitedIO] end   0 on DefaultDispatcher-worker-2
[limitedIO] end   4 on DefaultDispatcher-worker-5
[limitedIO] end   2 on DefaultDispatcher-worker-2


## 9) Structured concurrency in Kotlin

Structured concurrency is a design principle that organizes concurrent work into a tree of scopes so that lifetimes are explicit and bounded. In Kotlin coroutines, every coroutine runs in a CoroutineScope and has a parent Job. This gives you:
- Ownership and lifecycle: parents wait for children to finish (or get canceled) before completing.
- Cancellation propagation: canceling a parent cancels all its children.
- Exception propagation: failures in a child are reported to the parent (and typically cancel the family), unless you use supervisor semantics.

Why it matters:
- Prevents "fire-and-forget" leaks.
- Makes shutdown predictable: you can cancel a scope and know all work is stopped.
- Makes errors visible: failures bubble up instead of being silently lost.

### Core building blocks
- runBlocking: bridges blocking and suspending worlds; creates a root scope that completes when its child coroutines complete.
- coroutineScope: a suspending function that creates a child scope; it does not return until all its children are done or the scope fails.
- Job hierarchy: every coroutine has a Job that has a parent Job (except a root). Parents wait for children.

### Example A: Parents wait for children (coroutineScope)


In [13]:
runBlocking {
    val time = measureTimeMillis {
        coroutineScope { // this suspends until its children complete
            launch {
                delay(200)
                println("[structured] Child 1 done on ${Thread.currentThread().name}")
            }
            launch {
                delay(300)
                println("[structured] Child 2 done on ${Thread.currentThread().name}")
            }
            println("[structured] Inside coroutineScope")
        }
    }
    println("[structured] coroutineScope returned after ${time} ms (both children finished)")
}


[structured] Inside coroutineScope
[structured] Child 1 done on Execution of code 'import kotlinx.corou...'
[structured] Child 2 done on Execution of code 'import kotlinx.corou...'
[structured] coroutineScope returned after 306 ms (both children finished)


### Example B: Cancellation flows downward
If the parent scope is canceled, all children are canceled too.


In [14]:
runBlocking {
    val parent = launch {
        val c1 = launch {
            try { delay(1_000); println("[cancel-flow] c1 finished") } finally { println("[cancel-flow] c1 finally (canceled)") }
        }
        val c2 = launch {
            try { delay(1_000); println("[cancel-flow] c2 finished") } finally { println("[cancel-flow] c2 finally (canceled)") }
        }
        joinAll(c1, c2)
    }
    delay(200)
    println("[cancel-flow] Cancel parent")
    parent.cancelAndJoin()
    println("[cancel-flow] Parent joined (children were canceled)")
}


[cancel-flow] Cancel parent
[cancel-flow] c1 finally (canceled)
[cancel-flow] c2 finally (canceled)
[cancel-flow] Parent joined (children were canceled)


### Example C: Exception propagation in regular structured concurrency
By default, an uncaught exception in a child fails the parent and cancels siblings. Handle exceptions where appropriate, or use supervisor semantics when isolation is required.


In [15]:
runBlocking {
    try {
        coroutineScope {
            launch {
                delay(100)
                println("[exceptions] Child A failing")
                throw RuntimeException("Boom")
            }
            launch {
                try {
                    delay(500)
                    println("[exceptions] Child B finished (won't happen)")
                } finally {
                    println("[exceptions] Child B finally (canceled)")
                }
            }
        }
    } catch (e: Throwable) {
        println("[exceptions] Caught in parent: ${e::class.simpleName} -> ${e.message}")
    }
}


[exceptions] Child A failing
[exceptions] Child B finally (canceled)
[exceptions] Caught in parent: RuntimeException -> Boom


### Avoid GlobalScope for application code
GlobalScope.launch creates a top-level coroutine that isn’t bound to any lifecycle; it won’t be canceled when your feature/component is destroyed. Prefer passing/creating a CoroutineScope and launching in it. If you need siblings not to cancel each other, prefer supervisorScope or a CoroutineScope(SupervisorJob()). See the dedicated notebook: Coroutines/supervisor_job_and_scope.ipynb.

### Takeaways
- Always launch in a scope you control (lifecycle-owned scope).
- Use coroutineScope to group work and ensure completion.
- Cancellation flows downward; exceptions by default cancel the whole family.
- Use supervisorScope or SupervisorJob when sibling isolation is required (and attach a CoroutineExceptionHandler for visibility).


## 10) What is a Job in Kotlin coroutines?

A Job is the handle and lifecycle of a coroutine. When you call launch, it returns a Job. That Job lets you:
- Observe state: isActive, isCompleted, isCancelled
- Wait for completion: join()
- Cancel cooperatively: cancel() / cancelAndJoin()
- Participate in structured concurrency as part of a parent–child Job hierarchy

Key ideas:
- Lifecycle: New → Active → Completing → Completed (or Cancelling → Cancelled on cancellation/failure). You typically query via booleans rather than state enums.
- Hierarchy: Every coroutine has a Job; a parent Job waits for children and cancellation flows from parent to children.
- Job vs Deferred<T>: Deferred is a Job with a result (await()) and failure becomes an exception on await. A plain Job has no result (fire-and-forget but still structured).
- Start modes (advanced):
  - DEFAULT: start when scheduled.
  - LAZY: start only when needed (start(), join(), or await()).
  - ATOMIC: cannot be canceled before it starts executing the first instruction.
  - UNDISPATCHED: start running immediately in the current thread until the first suspension.

### Example 1: Create, observe, cancel, and join a Job


In [16]:
runBlocking {
    val job = launch {
        try {
            repeat(5) { i ->
                delay(150)
                println("[job] Tick $i on ${Thread.currentThread().name}")
            }
        } finally {
            println("[job] finally: cleanup before finish (canceled = ${!isActive})")
        }
    }
    println("[job] active? ${job.isActive}, completed? ${job.isCompleted}, cancelled? ${job.isCancelled}")
    delay(400)
    println("[job] Cancel it now")
    job.cancelAndJoin()
    println("[job] active? ${job.isActive}, completed? ${job.isCompleted}, cancelled? ${job.isCancelled}")
}


[job] active? true, completed? false, cancelled? false
[job] Tick 0 on Execution of code 'import kotlinx.corou...'
[job] Tick 1 on Execution of code 'import kotlinx.corou...'
[job] Cancel it now
[job] finally: cleanup before finish (canceled = true)
[job] active? false, completed? true, cancelled? true


### Example 2: Job vs Deferred<T>
Deferred is a Job with a result. You can still cancel/join, but you also have await() to get the value (or exception).


In [17]:
runBlocking {
    val deferred: Deferred<Int> = async {
        delay(200)
        21 + 21
    }
    println("[deferred] isActive=${deferred.isActive}")
    val value = deferred.await() // suspends until ready
    println("[deferred] value=$value, isCompleted=${deferred.isCompleted}")
}


[deferred] isActive=true
[deferred] value=42, isCompleted=true


### Example 3: Parent–child Jobs and cancellation propagation
Canceling the parent cancels all its children. Parents don’t complete until children complete.


In [18]:
runBlocking {
    val parent: Job = launch {
        val c1 = launch {
            try { delay(1_000); println("[family] c1 finished") } finally { println("[family] c1 finally (canceled=${!isActive})") }
        }
        val c2 = launch {
            try { delay(1_000); println("[family] c2 finished") } finally { println("[family] c2 finally (canceled=${!isActive})") }
        }
        joinAll(c1, c2)
        println("[family] parent body finished (children done)")
    }
    delay(250)
    println("[family] Cancel parent")
    parent.cancelAndJoin()
    println("[family] Parent joined")
}


[family] Cancel parent
[family] c1 finally (canceled=true)
[family] c2 finally (canceled=true)
[family] Parent joined


Takeaways:
- Job is the lifecycle handle of a coroutine: cancel, join, and inspect it.
- Prefer launching coroutines inside a scope you control so Jobs form a clear tree (structured concurrency).
- Use Deferred when you need a result; otherwise, a plain Job is enough.


## 11) Dispatcher.Main.immediate: what it is and when to use it

Dispatcher.Main.immediate is a variant of the main-thread dispatcher available on UI runtimes (Android, Swing, JavaFX). It behaves like Dispatchers.Main, but with one important difference:

- If you are already running on the Main thread, it will execute the coroutine continuation immediately (without an extra dispatch/post to the event loop).
- If you are not on the Main thread, it will dispatch to the Main thread as usual.

This can remove an extra scheduling hop and reduce latency in hot UI paths.

Important:
- Requires the appropriate runtime artifact so that Dispatchers.Main is available:
  - Android: add dependency on kotlinx-coroutines-android
  - Swing: add dependency on kotlinx-coroutines-swing
  - JavaFX: add dependency on kotlinx-coroutines-javafx
- In this notebook (plain JVM), Dispatchers.Main is not available, so the examples below are illustrative and not meant to be executed here.

How it differs from related options:
- vs Dispatchers.Main: Main always posts/dispatches to the event loop even if you are already on Main, whereas Main.immediate runs inline in that case.
- vs Dispatchers.Unconfined: Unconfined is not confined to any thread and may resume on different threads after suspension; Main.immediate is still strictly confined to the Main UI thread, it just avoids an extra dispatch when already there.

When to use:
- UI event handlers or collectors already running on the Main thread where you want to update UI state immediately without the cost of an extra dispatch.
- Hot paths where a double-dispatch could cause jank (e.g., animation or tight render loops).

Caveats:
- Re-entrancy: because it may run inline, code can execute sooner than expected and re-enter current call stacks. Ensure your UI/state logic is safe to run re-entrantly.
- Don’t use it to “skip” correctness checks; it doesn’t bypass cancellation or structured concurrency rules.

Illustrative examples (do not run in this notebook):

```kotlin
// Android example (requires kotlinx-coroutines-android)
class MyViewModel : ViewModel() {
    private val vmScope = viewModelScope // already on Main by default

    fun onButtonClick() {
        // Will execute inline if on Main, or dispatch to Main otherwise
        vmScope.launch(Dispatchers.Main.immediate) {
            // Update UI state right away
            _uiState.value = _uiState.value.copy(loading = true)

            // Perform work off main, then come back
            val data = withContext(Dispatchers.IO) { repo.load() }
            _uiState.value = _uiState.value.copy(loading = false, data = data)
        }
    }
}
```

```kotlin
// Swing example (requires kotlinx-coroutines-swing)
val uiScope = CoroutineScope(SupervisorJob() + Dispatchers.Main)

fun onMenuAction() {
    // If this runs on the EDT already, code inside runs immediately
    uiScope.launch(Dispatchers.Main.immediate) {
        statusLabel.text = "Working..."
        val result = withContext(Dispatchers.IO) { doBlockingWork() }
        statusLabel.text = "Done: $result"
    }
}
```

Takeaways:
- Prefer Dispatchers.Main for general UI code; switch to Dispatchers.Main.immediate when you specifically want to avoid the extra dispatch while still staying on Main.
- Test for re-entrancy hazards when adopting Main.immediate in complex UI flows.



## Appendix: The Android event loop (Looper/MessageQueue) and coroutines

This notebook runs on the plain JVM, but when we talk about Dispatchers.Main (and Dispatchers.Main.immediate) we implicitly refer to the Android main thread’s event loop. Here’s what that means and why it matters.

- Main thread and Looper:
  - Android creates a single UI thread (often called the Main thread). That thread runs a Looper, which repeatedly pulls tasks from a MessageQueue and executes them.
  - Apps enqueue work to that queue via Handler (legacy) or higher-level APIs (e.g., View.post, Activity.runOnUiThread, lifecycleScope.launch on Main, etc.).

- Message vs Runnable and posting:
  - A Handler associated with Looper.getMainLooper() can post Runnables or send Messages to the queue.
  - Posting schedules the work to run “later” on the main thread in FIFO order relative to other posted items at the same time slice.

- Choreographer and frames (why latency matters):
  - Rendering is paced by Choreographer, which schedules frame callbacks (input, animation, traversal, draw) roughly every 16.6 ms on 60 Hz devices.
  - Extra hops (additional posts to the queue) can push UI work to the next frame, causing visible jank. Minimizing unnecessary dispatches helps keep updates within the same frame.

- How coroutines reach Android’s main thread:
  - Dispatchers.Main on Android is implemented via a Handler bound to Looper.getMainLooper() (from kotlinx-coroutines-android).
  - Launching or resuming a coroutine on Dispatchers.Main posts its continuation to the main MessageQueue.

- What Dispatchers.Main.immediate changes:
  - If your coroutine is already running on the main thread, Main.immediate resumes the continuation immediately (inline), instead of posting another Runnable to the queue.
  - If you’re not on main, it behaves like Dispatchers.Main and posts to the main queue.
  - Benefit: avoids one extra scheduling hop, improving latency on hot UI paths.
  - Caveat: re-entrancy — inline execution can re-enter current call stacks sooner than expected. Ensure your logic is safe for this.

- Ordering and fairness notes:
  - A normal post goes behind already-queued work for that Looper, so it might run in a later turn of the event loop.
  - Main.immediate, when already on main, runs now, which can change the relative ordering compared to code that expects a posted turn. Use intentionally and document assumptions.

Illustrative snippets (do not run in this notebook):

```kotlin
// 1) Posting work to the Android main thread using a Handler
val mainHandler = Handler(Looper.getMainLooper())
mainHandler.post {
    // Runs on the main thread in a future loop turn
    textView.text = "Hello"
}
```

```kotlin
// 2) Coroutines: Dispatchers.Main vs Main.immediate
class MyViewModel : ViewModel() {
    fun onClick() {
        // Both target the main thread; immediate avoids an extra post if already on main
        viewModelScope.launch(Dispatchers.Main) {
            // Will be posted to the queue even if we're already on main
            _uiState.value = _uiState.value.copy(loading = true)
        }
        viewModelScope.launch(Dispatchers.Main.immediate) {
            // Will run inline if we're already on main
            _uiState.value = _uiState.value.copy(loading = true)
        }
    }
}
```

```kotlin
// 3) Frame timing with Choreographer (conceptual example)
Choreographer.getInstance().postFrameCallback { frameTimeNanos ->
    // Called every frame; extra dispatches can push this work to the next frame
    // Prefer immediate updates when already on main to avoid extra latency
}
```

Takeaways:
- The Android main thread processes a queue via Looper/MessageQueue; posting adds a hop.
- Dispatchers.Main posts coroutine continuations to that queue.
- Dispatchers.Main.immediate skips the extra post when already on main, reducing latency but introducing re-entrancy considerations.
